# RSS скрапинг сайта investing.com

In [ ]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [ ]:
url = "https://ru.investing.com/webmaster-tools/rss"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Найти заголовок "Новости"
news_section = soup.find('h2', string='Новости')

if news_section:
    # Найти родительский div с нужными классами
    rss_column = news_section.find_parent('div', class_='rssColumn halfSizeColumn float_lang_base_2')

    if rss_column:
        # Найти ul с классом "rssBox" внутри этого div
        rss_box = rss_column.find('ul', class_='rssBox')

        if rss_box:
            # Найти все элементы <li> внутри ul
            list_items = rss_box.find_all('li')

            # Извлечь ссылки из каждого li и отфильтровать по расширению ".rss"
            rss_links = []
            for item in list_items:
                a_tag = item.find('a')
                if a_tag and a_tag.get('href') and a_tag['href'].endswith('.rss'):
                    rss_links.append(a_tag['href'])

            # Вывести результаты
            for link in rss_links:
                print(link)
        else:
            print("Список RSS не найден.")
    else:
        print("Контейнер с новостями не найден.")
else:
    print("Раздел 'Новости' не найден.")


https://ru.investing.com/rss/news.rss
https://ru.investing.com/rss/news_1061.rss
https://ru.investing.com/rss/news_1062.rss
https://ru.investing.com/rss/news_1063.rss
https://ru.investing.com/rss/news_1064.rss
https://ru.investing.com/rss/news_1065.rss
https://ru.investing.com/rss/news_301.rss
https://ru.investing.com/rss/news_356.rss
https://ru.investing.com/rss/news_357.rss
https://ru.investing.com/rss/news_462.rss
https://ru.investing.com/rss/news_1.rss
https://ru.investing.com/rss/news_285.rss
https://ru.investing.com/rss/news_477.rss
https://ru.investing.com/rss/news_11.rss
https://ru.investing.com/rss/news_25.rss
https://ru.investing.com/rss/news_95.rss
https://ru.investing.com/rss/news_12.rss
https://ru.investing.com/rss/news_14.rss
https://ru.investing.com/rss/news_287.rss


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

url = "https://ru.investing.com/webmaster-tools/rss"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Найти заголовок "Новости"
# news_section = soup.find('h2', text='Новости')
news_section = soup.find('h2', string='Новости')

if news_section:
    # Найти родительский div с нужными классами
    rss_column = news_section.find_parent('div', class_='rssColumn halfSizeColumn float_lang_base_2')

    if rss_column:
        # Найти ul с классом "rssBox" внутри этого div
        rss_box = rss_column.find('ul', class_='rssBox')

        if rss_box:
            # Найти все элементы <li> внутри ul
            list_items = rss_box.find_all('li')

            # Извлечь ссылки и создать список для хранения данных
            data = []
            for item in list_items:
                a_tag = item.find('a')
                if a_tag and a_tag.get('href') and a_tag['href'].endswith('.rss'):
                    link = a_tag['href']

                    # Получить содержимое страницы RSS
                    rss_response = requests.get(link)
                    rss_soup = BeautifulSoup(rss_response.text, 'xml')

                    # Извлечь данные из XML
                    try:
                        pub_date_tag = rss_soup.find('pubDate')
                        channel_title_tag = rss_soup.find('channel').find('title')
                        if not pub_date_tag or not channel_title_tag:
                            raise ValueError("Нет pubDate или channel/title")
                        pub_date = pub_date_tag.text
                        channel_title = channel_title_tag.text
                        item_title = a_tag.text

                        # Преобразовать дату в нужный формат
                        # date_obj = datetime.strptime(pub_date, '%a %b %d %H:%M:%S %Y')
                        # formatted_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
                        date_obj = datetime.strptime(pub_date, '%Y-%m-%d %H:%M:%S')
                        formatted_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')

                        data.append([formatted_date, channel_title, link, item_title])
                    except Exception as e:
                        print(f"Ошибка при обработке RSS-ссылки {link}: {e}")

            # Создать DataFrame из собранных данных
            df = pd.DataFrame(data, columns=["Дата", "Раздел", "Ссылка", "Заголовок"])

            # Вывести DataFrame
            print(df)
        else:
            print("Список RSS не найден.")
    else:
        print("Контейнер с новостями не найден.")
else:
    print("Раздел 'Новости' не найден.")


Ошибка при обработке RSS-ссылки https://ru.investing.com/rss/news_462.rss: 'NoneType' object has no attribute 'find'
Ошибка при обработке RSS-ссылки https://ru.investing.com/rss/news_477.rss: 'NoneType' object has no attribute 'find'
                   Дата                                 Раздел  \
0   2025-06-24 20:25:21                            Все Новости   
1   2025-06-24 16:46:29           Рейтинги биржевых аналитиков   
2   2025-06-24 20:21:03    Отчеты о прибылях и убытках и слухи   
3   2025-06-24 18:09:28       Расшифровки по отчетам о доходах   
4   2025-06-24 20:21:32                              Формы SEC   
5   2025-06-24 11:06:31  Идеи для инвестиций на фондовом рынке   
6   2025-06-24 16:46:19                    Новости криптовалют   
7   2025-06-24 20:25:21                       Новости компаний   
8   2025-06-24 20:20:09            Новости инсайдерских торгов   
9   2025-06-24 08:47:39                Новости валютного рынка   
10  2025-06-24 12:10:19                 